In [1]:
from short_circuit import transform_to_abc, PolarRepresentation

# Unsymmetrical Faults

From: *Glover, J. D., Sarma, M. S., & Overbye, T. (2022). Power System Analysis and Design, SI Edition. Chapter 10 - Unsymmetrical Faults*

The following assumptions are made:
1. The power system operates under balanced steady-state conditions before the fault occurs. Thus, the zero-, positive-, and negative-sequence networks are uncoupled before the fault occurs. During unsymmetrical faults, they are interconnected only at the fault location.
2. Prefault load current is neglected. Because of this, the positive-sequence internal voltages of all machines are equal to the prefault voltage.
3. Transformer winding resistances and shunt admittances are neglected.
4. Transmission-line series resistances and shunt admittances are neglected.
5. Synchronous machine armature resistance, saliency, and saturation are neglected.
6. All nonrotating impedance loads are neglected.
7. Induction motors are either neglected (especially for motors rated 50 hp or less) or represented in the same manner as synchronous machines.

## Single-line diagram

<img src="./figures/ex3_line_diagram.png" style="max-width: 50%; height: auto; margin-left:auto; margin-right:auto">

## Per-Unit Systems

In [2]:
from short_circuit import Quantity, PerUnitSystem
Q_ = Quantity

The power system has a high and low voltage level:

In [3]:
pusys_LV = PerUnitSystem(S_base=Q_(100, "MVA"), U_base=Q_(13.8, "kV"))
pusys_HV = PerUnitSystem(S_base=Q_(100, "MVA"), U_base=Q_(138.0, "kV"))

Sequence impedances of the transmission line in per-unit:

In [4]:
X1_line = X2_line = pusys_HV.get_per_unit_impedance(Q_(20, 'ohm'))
X0_line = pusys_HV.get_per_unit_impedance(Q_(60, 'ohm'))
X0_line, X1_line, X2_line

(0.315059861373661, 0.10501995379122034, 0.10501995379122034)

## Sequence Networks

<img src="./figures/ex3_sequence_networks.png" style="max-width: 50%; height: auto; margin-left:auto; margin-right:auto">

In [5]:
from short_circuit import Network

> **Note !**<br>
> - Branches which are connected at one end to the reference node (ground), must always have the reference node as their start node!
> - If a branch contains a voltage source (synchronous machine), this must be indicated by setting parameter `has_source` to True.

### Zero-Sequence Network

In [6]:
nw0 = Network()

nw0.add_branch(0.05j, end_node_ID="1")
nw0.add_branch(0.25j, end_node_ID="2")

nw0.show_impedance_matrix()

            1           2
1  0.00+0.05j  0.00+0.00j
2  0.00+0.00j  0.00+0.25j


### Positive-Sequence Network

In [7]:
nw1 = Network()

nw1.add_branch(0.15j, end_node_ID="1", has_source=True)
nw1.add_branch(0.2j, end_node_ID="2", has_source=True)
nw1.add_branch(0.305j, start_node_ID="1", end_node_ID="2")

nw1.show_impedance_matrix()

                    1                   2
1  0.000000+0.115649j  0.000000+0.045802j
2  0.000000+0.045802j  0.000000+0.138931j


### Negative-Sequence Network

In [8]:
nw2 = Network()

nw2.add_branch(0.17j, end_node_ID="1")
nw2.add_branch(0.21j, end_node_ID="2")
nw2.add_branch(0.305j, start_node_ID="1", end_node_ID="2")

nw2.show_impedance_matrix()

                    1                   2
1  0.000000+0.127810j  0.000000+0.052117j
2  0.000000+0.052117j  0.000000+0.145620j


## Three-Phase Short-Circuit

*Calculate the per-unit subtransient fault currents in phases $a$, $b$, and $c$ for a bolted three-phase-to-ground short circuit at bus 2.*

In [9]:
from short_circuit import ThreePhaseFault

> The fault currents are balanced and have only a positive-sequence component. Therefore, we only need the positive-sequence network when calculating three-phase fault currents.

In [10]:
fault = ThreePhaseFault(nw1, U_prefault=1.05)
fault.set_faulted_node("2")

In [11]:
If_1_pu = fault.get_fault_current()
print(f"{If_1_pu:.3f}")

0.000-7.558j


The zero-sequence current and negative-sequence current are both zero. To get the three-phase current in complex polar form:

In [12]:
If_abc_pu = transform_to_abc([0.0, If_1_pu, 0.0])
print(PolarRepresentation.from_complex_vector(If_abc_pu))

[7.5577 < -90.00°, 7.5577 < 150.00°, 7.5577 < 30.00°]


## Single Line-to-Ground Fault

*Calculate the subtransient fault current in per-unit and in kA for a bolted single line-to-ground short circuit from phase $a$ to ground at bus 2. Also, calculate the per-unit line-to-ground voltages at faulted bus 2*

In [13]:
from short_circuit import LineToGroundFault

In [14]:
fault = LineToGroundFault([nw0, nw1, nw2], U_prefault=1.05)
fault.set_faulted_node("2")

**Subtransient fault current**

Sequence components of the fault current in per-unit:

In [15]:
If_012_pu = fault.get_fault_current_012()
print(f"{PolarRepresentation.from_complex_vector(If_012_pu)} pu")

[1.9643 < -90.00°, 1.9643 < -90.00°, 1.9643 < -90.00°] pu


Three-phase fault current in per-unit:

In [16]:
If_abc_pu = fault.get_fault_current_abc()
print(f"{PolarRepresentation.from_complex_vector(If_abc_pu)} pu")

[5.8928 < -90.00°, 0.0000 < 18.43°, 0.0000 < 18.43°] pu


Three-phase fault current in kA:

In [17]:
If_abc = pusys_LV.get_actual_current(If_abc_pu)
print(f"{PolarRepresentation.from_complex_quantity(If_abc.to('kA'))} kA")

[24.6536 kA < -90.00°, 0.0000 kA < 18.43°, 0.0000 kA < 18.43°] kA


> Note that the `PerUnitSystem` object `pusys_LV` returns the current as a Pint `Quantity` object.

**Line-to-ground voltages at faulted bus**

Sequence components of the voltages at the fault:

In [18]:
Uf_012_pu = fault.get_node_voltage_012("2")
print(f"{PolarRepresentation.from_complex_vector(Uf_012_pu)} pu")

[0.4911 < 180.00°, 0.7771 < 0.00°, 0.2860 < 180.00°] pu


Transforming to the phase domain, the line-to-ground voltages at faulted bus 2 are:

In [19]:
Uf_abc_pu = fault.get_node_voltage_abc("2")
print(f"{PolarRepresentation.from_complex_vector(Uf_abc_pu)} pu")

[0.0000 < 0.00°, 1.1791 < -128.66°, 1.1791 < 128.66°] pu


Note that for the single line-to-ground fault on phase $a$ at bus 2, there is an overvoltage of 1.179 per unit on unfaulted phases $b$ and $c$.

## Line-to-Line Fault

*Calculate the subtransient fault current in per-unit and in kA for a bolted line-to-line fault from phase $b$ to $c$ at bus 2.*

In [20]:
from short_circuit import LineToLineFault

In [21]:
fault = LineToLineFault([nw0, nw1, nw2], U_prefault=1.05)
fault.set_faulted_node("2")

Sequence components of the fault current in per-unit:

In [22]:
If_012_pu = fault.get_fault_current_012()
print(f"{PolarRepresentation.from_complex_vector(If_012_pu)} pu")

[0.0000 < 0.00°, 3.6900 < -90.00°, 3.6900 < 90.00°] pu


Three-phase fault current in per-unit:

In [23]:
If_abc_pu = fault.get_fault_current_abc()
print(f"{PolarRepresentation.from_complex_vector(If_abc_pu)} pu")

[0.0000 < 0.00°, 6.3913 < 180.00°, 6.3913 < -0.00°] pu


Three-phase fault current in kA:

In [24]:
If_abc = pusys_LV.get_actual_current(If_abc_pu)
print(f"{PolarRepresentation.from_complex_quantity(If_abc.to('kA'))} kA")

[0.0000 kA < 0.00°, 26.7392 kA < 180.00°, 26.7392 kA < -0.00°] kA


> Note that the `PerUnitSystem` object `pusys_LV` returns the current as a Pint `Quantity` object.

## Double Line-to-Ground Fault

*Calculate (a) the subtransient fault current in each phase, (b) neutral fault current, and (c) contributions to the fault current from the motor and from the transmission line, for a bolted double line-to-ground fault from phase $b$ to $c$ to ground at bus 2. The $\Delta$-Y transformator phase shifts are neglected.*

In [25]:
from short_circuit import DoubleLineToGroundFault

In [26]:
fault = DoubleLineToGroundFault([nw0, nw1, nw2], U_prefault=1.05)
fault.set_faulted_node("2")

**Subtransient fault current**

Sequence components of the fault current in per-unit:

In [27]:
If_012_pu = fault.get_fault_current_012()
print(f"{PolarRepresentation.from_complex_vector(If_012_pu)} pu")

[1.6734 < 90.00°, 4.5464 < -90.00°, 2.8730 < 90.00°] pu


Three-phase fault current in per-unit:

In [28]:
If_abc_pu = fault.get_fault_current_abc()
print(f"{PolarRepresentation.from_complex_vector(If_abc_pu)} pu")

[0.0000 < 0.00°, 6.8983 < 158.66°, 6.8983 < 21.34°] pu


Three-phase fault current in kA:

In [29]:
If_abc = pusys_LV.get_actual_current(If_abc_pu)
print(f"{PolarRepresentation.from_complex_quantity(If_abc.to('kA'))} kA")

[0.0000 kA < 0.00°, 28.8603 kA < 158.66°, 28.8603 kA < 21.34°] kA


**Neutral fault current**

Neutral fault current in kA:

In [30]:
In_abc = If_abc[1] + If_abc[2]
print(f"{PolarRepresentation.from_complex_quantity(In_abc.to('kA'))} kA")

21.0036 kA < 90.00° kA


**Contributions to the fault current from the motor and from the transmission line**

The transmission line is the network branch between node 1 and node 2.

In [31]:
I_line_012_pu = fault.get_branch_current_012(("1", "2"))
print(f"{PolarRepresentation.from_complex_vector(I_line_012_pu)} pu")

I_line_abc_pu = fault.get_branch_current_abc(("1", "2"))
print(f"{PolarRepresentation.from_complex_vector(I_line_abc_pu)} pu")

I_line_abc = pusys_HV.get_actual_current(I_line_abc_pu)
print(f"{PolarRepresentation.from_complex_quantity(I_line_abc.to('kA'))} kA")

[0.0000 < 0.00°, 1.3882 < -90.00°, 0.8808 < 90.00°] pu
[0.5075 < -90.00°, 1.9813 < 172.64°, 1.9813 < 7.36°] pu
[0.2123 kA < -90.00°, 0.8289 kA < 172.64°, 0.8289 kA < 7.36°] kA


The motor is in the branch between the reference node (ground) and node 2. 

In [32]:
I_motor_012_pu = fault.get_branch_current_012(("ref", "2"))
print(f"{PolarRepresentation.from_complex_vector(I_motor_012_pu)} pu")

I_motor_abc_pu = fault.get_branch_current_abc(("ref", "2"))
print(f"{PolarRepresentation.from_complex_vector(I_motor_abc_pu)} pu")

I_motor_abc = pusys_LV.get_actual_current(I_motor_abc_pu)
print(f"{PolarRepresentation.from_complex_quantity(I_motor_abc.to('kA'))} kA")

[1.6734 < 90.00°, 3.1582 < -90.00°, 1.9922 < 90.00°] pu
[0.5075 < 90.00°, 4.9986 < 153.17°, 4.9986 < 26.83°] pu
[2.1230 kA < 90.00°, 20.9128 kA < 153.17°, 20.9128 kA < 26.83°] kA


Alternative way to determine the contributions from the transmission line and the motor:

In [33]:
Ix2_012 = fault.get_currents_to_node_012("2")
# Returns the currents from all branches between any node `x` and node 2.

for x, I_012 in Ix2_012:
    I_abc = transform_to_abc(I_012)
    print(
        f"sequence components of current from node '{x}': "
        f"{PolarRepresentation.from_complex_vector(I_012)} pu"
    )
    print(
        f"three-phase current from node '{x}': "
        f"{PolarRepresentation.from_complex_vector(I_abc)} pu"
    )
    print()

sequence components of current from node 'ref': [1.6734 < 90.00°, 3.1582 < -90.00°, 1.9922 < 90.00°] pu
three-phase current from node 'ref': [0.5075 < 90.00°, 4.9986 < 153.17°, 4.9986 < 26.83°] pu

sequence components of current from node '1': [0.0000 < 0.00°, 1.3882 < -90.00°, 0.8808 < 90.00°] pu
three-phase current from node '1': [0.5075 < -90.00°, 1.9813 < 172.64°, 1.9813 < 7.36°] pu



### Including $\Delta$-Y Transformer Shift

In accordance with the American standard, positive-sequence quantities on the high-voltage side of the transformers lead their corresponding quantities on the low-voltage side by 30°. Also, the negative-sequence phase shifts are the reverse of the positive-sequence phase shifts.

$\Delta$-Y transformer phase shifts have no effect on the fault currents and no effect on the contribution to the fault currents on the fault side of the $\Delta$-Y transformer. However, on the other side of the $\Delta$-Y transformer, the positive- and negative-sequence components of the contributions to the fault currents are shifted by +/- 30°, which affects both the magnitude as well as the angle of the phase components of these fault contributions for unsymmetrical faults.

In [34]:
from short_circuit import add_deltastar_transformer_shift

In [35]:
I_line_012_pu = add_deltastar_transformer_shift(I_line_012_pu)
print(f"{PolarRepresentation.from_complex_vector(I_line_012_pu)}")

[0.0000 < 0.00°, 1.3882 < -60.00°, 0.8808 < 60.00°]


In [36]:
I_line_abc_pu = transform_to_abc(I_line_012_pu)
print(f"{PolarRepresentation.from_complex_vector(I_line_abc_pu)}")

[1.2166 < -21.17°, 2.2690 < 180.00°, 1.2166 < 21.17°]


In [37]:
I_line_abc = pusys_HV.get_actual_current(I_line_abc_pu)
print(f"{PolarRepresentation.from_complex_quantity(I_line_abc.to('kA'))}")

[0.5090 kA < -21.17°, 0.9493 kA < 180.00°, 0.5090 kA < 21.17°]
